In [1]:
%load_ext autoreload
%autoreload 2
%autosave 60
%matplotlib inline
from ivanocode.ivanocommon import *
import pandas as pd

Autosaving every 60 seconds


In [2]:
raw = 'raw'
submissions = 'submissions'

In [3]:
df_sample_submission = pd.read_csv(f'{raw}/sample_submission.csv')
df_sample_submission.columns

Index(['id', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28'],
      dtype='object')

In [4]:
df_sales_train_melt = pd.read_feather('df_sales_train_melt.feather')

In [5]:
df_sales_train_melt.head(1)

,id,item_id,dept_id,cat_id,store_id,state_id,day_id,sales,daily_avg_count,monthly_avg_count,day_date,day_date_str,month_id,month,year,date,wm_yr_wk,sell_price,sell_price_flip_count
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,0,0.313643,9.409305,2011-01-29,2011-01-29,1,1,2011,2011-01-29,11101,NaN,1


In [6]:
days = df_sales_train_melt['day_id'].unique()
days[0], days[-1]

(1, 1913)

In [7]:
# F1 = d_1914, ... F28 = d_1941

In [8]:
query = ("id == 'HOBBIES_1_001_CA_1_validation' or "+
         "id == 'HOBBIES_1_002_CA_1_validation' or "+
         "id == 'HOBBIES_1_004_CA_1_validation'")
subset = df_sales_train_melt.query(query).copy()

In [9]:
%%time
min_day = 1885
df_submission_melt = (df_sales_train_melt
    .groupby('id', as_index=False)
    .tail(28)
)

df_submission = (df_submission_melt
    .pivot(index='id', columns='day_id', values='sales')
    .rename(lambda x: f'F{x-min_day}', axis=1)
    .reset_index()
    .sort_values('id')
)

CPU times: user 6.26 s, sys: 2.6 s, total: 8.86 s
Wall time: 9.05 s


In [10]:
df_train = df_sales_train_melt.query('day_id < 1886').copy()

In [11]:
df_pred = df_train.query(f'day_id >= {1886-28} & day_id <= {1913-28}').copy()

In [12]:
df_valid = df_sales_train_melt.query('day_id >= 1886 & day_id <= 1913').copy()

In [13]:
aggregation_levels = {
    11: ['item_id', 'state_id'],
    10: ['item_id'],
    9:  ['store_id', 'dept_id'],
    8:  ['store_id', 'cat_id'],
    7:  ['state_id', 'dept_id'],
    6:  ['state_id', 'cat_id'],
    5:  ['dept_id'],
    4:  ['cat_id'],
    3:  ['store_id'],
    2:  ['state_id'],
    1:  [],
}
agg_level_expected_counts = {
    11: 9147,
    10: 3049,
    9:  70,
    8:  30,
    7:  21,
    6:  9,
    5:  7,
    4:  3,
    3:  10,
    2:  3
}

In [104]:
def add_group_id_col(df, grp_id, grp_fields):
    df['id'] = str(grp_id)
    if len(grp_fields) > 0:
        # Note to self: categoricals require cast
        for col in grp_fields:
            df['id'] = df['id'].str.cat(df[col], sep=':')

def with_aggregates(df):
    result = [df]
    df['agg_level'] = 12
    df['sales_$'] = df['sales']*df['sell_price']
    for grp_id, grp_fields in aggregation_levels.items():
        grp_11 = (df
                  .groupby(grp_fields + ['day_id'], as_index=False)
                  .agg({'sales': 'sum', 'sales_$': 'sum'})
        )
        grp_11['agg_level'] = grp_id
        add_group_id_col(grp_11, grp_id, grp_fields)

        result.append(grp_11)

    df = pd.concat(result, sort=False)
    df['sales_delta_sum'] = (df
              .groupby(['id'], as_index=False)
              ['sales']
              .transform(lambda x: x.diff().abs().sum())
    )
    # Note to self: ordering dependencies are a beast! Wouldn't it be fun to prevent cell & statement reordering unless it's safe & consistent?
    df['day_id_rel'] = df['day_id'] - df['day_id'].min()

    return df

In [105]:
df_valid_w_aggs = with_aggregates(df_valid)

In [106]:
df_valid_w_aggs.tail(1)

,id,item_id,dept_id,cat_id,store_id,state_id,day_id,sales,daily_avg_count,monthly_avg_count,...,month,year,date,wm_yr_wk,sell_price,sell_price_flip_count,sales_$,day_id_rel,agg_level,sales_delta_sum
27,1,NaN,NaN,NaN,NaN,NaN,1913,49795,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,162924.07,27,1,133244.0


In [107]:
df_pred_w_aggs = with_aggregates(df_pred)

In [108]:
df_pred_w_aggs.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,day_id,sales,daily_avg_count,monthly_avg_count,...,month,year,date,wm_yr_wk,sell_price,sell_price_flip_count,day_id_rel,sales_$,agg_level,sales_delta_sum
56619930,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1858,0,0.313643,9.409305,...,2.0,2016.0,2016-02-29,11605.0,8.26,1.0,0,0.0,12,27.0
56619931,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1858,0,0.257710,7.731312,...,2.0,2016.0,2016-02-29,11605.0,3.97,1.0,0,0.0,12,9.0
56619932,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1858,0,0.150549,4.516466,...,2.0,2016.0,2016-02-29,11605.0,2.97,1.0,0,0.0,12,20.0


In [117]:
cols = ['id', 'day_id_rel', 'sales', 'sales_$', 'agg_level']
t = (df_valid_w_aggs[cols]
        .merge(df_pred_w_aggs[cols], 
               on=['id', 'day_id_rel'], 
               suffixes=('_valid', '_pred')))
t['daily_sales_err^2'] = (t['sales_valid'] - t['sales_pred']).pow(2)
# t['sales_delta_sum'] = (t
#           .groupby(['id'], as_index=False)
#           ['sales_valid']
#           .transform(lambda x: x.diff().abs().sum())
# )
# t['daily_sales_err^2_sum'] = t.groupby('id')['daily_sales_err^2'].transform(lambda x: x.sum())
def agg_as_is(x):
    vals = x.unique()
    if (len(vals) == 1):
        return vals[0]
    raise ValueError(f"Expected single unique val, got: {vals}" )

t = t.groupby(['id', 'agg_level_valid'], as_index=False).agg({
    'sales_valid': lambda x: x.diff().pow(2).sum(),
    'daily_sales_err^2': lambda x: x.sum(),
    'sales_$_valid': 'sum'
}).rename({
    'sales_valid': 'sales_diff^2_sum',
    'daily_sales_err^2': 'sales_err^2_sum'
}, axis=1)
t['agg_weight'] = t.groupby('agg_level_valid')['sales_$_valid'].transform(lambda x: x.sum())
t['series_weight'] = t['sales_$_valid']/t['agg_weight']

In [126]:
h = 28
n = 28 # TODO: this should come from training, not validation
t['rmsse'] = ((n-1)/float(h)*t['sales_err^2_sum']/t['sales_diff^2_sum']).pow(0.5)
# TODO: google rmsse est for series with no sales
t.loc[t['sales_$_valid'] ==0, 'rmsse'] = 0
t['wrmsse'] = t['rmsse']*t['series_weight']

In [129]:
# that's underestimation of 0.83770 from the submission, plus I've got a bunch of todos above to address
wrmsse_total = t['wrmsse'].sum()/12
wrmsse_total

0.7456401780934122

In [99]:
t.query('id == "HOBBIES_1_001_CA_1_validation"')

,id,sales_diff^2_sum,sales_err^2_sum,sales_$_valid,rmsse
26720,HOBBIES_1_001_CA_1_validation,68,73,224.94,1.017442


In [ ]:
# TODO: for each of the agg levels:
# calculate sum of everything on that level, divide sales_$_valid by that to get weight
# multiply rmsse by that & sum the thing
# divide the sum by the number of agg level

In [ ]:
t.head(1)

In [15]:
agg_series_counts = (df_valid_w_aggs.query('day_id == 1886')
    [['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'agg_level', 'state_id']]
    .groupby(['agg_level'], as_index=False)
    .count()
)

In [16]:
df_valid_w_aggs.query('day_id==1909')['id'].count()

42840

In [17]:
grp_fields=['state_id', 'cat_id']
denom = lambda x: x.diff().abs().sum()
denom.__name__ = 'a'
grp_11 = (df_valid
    .groupby(grp_fields + ['day_id'], as_index=False)
    .agg({'sales': 'sum', 'sales_$': 'sum'})
)
grp_11.head()

,state_id,cat_id,day_id,sales,sales_$
0,CA,FOODS,1886,9821,27349.95
1,CA,FOODS,1887,9845,26741.79
2,CA,FOODS,1888,9165,24731.91
3,CA,FOODS,1889,9941,26434.36
4,CA,FOODS,1890,11891,32464.87


In [18]:
pd.__version__

'0.25.0'

In [19]:
df_valid.query('id == "HOBBIES_1_001_CA_1_validation"')['sales'].diff().abs().sum()

30.0

In [20]:
%%time
for level, expected_count in agg_level_expected_counts.items():
    actual_count = (agg_series_counts
         .query(f'agg_level == {level}')
         .drop('agg_level', axis=1)
         .max(axis=1)
         .values[0]
    )
    assert expected_count == actual_count, f"Expected {expected_count} at level {level}, got: {actual_count}"

CPU times: user 38.9 ms, sys: 1.59 ms, total: 40.5 ms
Wall time: 40.5 ms


In [21]:
# grp_11['id'] = grp_11.sum(axis=1) # fishy, ints and not they sorta collide
"""
>>> df_valid_w_aggs.query('day_id == 1886')['id'].value_counts()[:3]
1897    3041
1898    1952
1899    1394
Name: id, dtype: int64
"""

"\n>>> df_valid_w_aggs.query('day_id == 1886')['id'].value_counts()[:3]\n1897    3041\n1898    1952\n1899    1394\nName: id, dtype: int64\n"

In [22]:
# TODO:
# pre-aggregate series
# add weight per sales volumes
# calculate rmssse
# calculate wrmsse
# ...
# Profit!

In [23]:
eval_rows = df_sample_submission[df_sample_submission['id'].str.contains('_evaluation')]

In [24]:
(pd.concat([df_submission, eval_rows])
    .to_csv(f'{submissions}/0200-naive.csv', index=False)
)

In [25]:
!head -n2 {submissions}/0200-naive.csv

id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
FOODS_1_001_CA_1_validation,2,1,1,0,4,0,0,4,1,3,0,1,0,2,2,0,1,1,0,2,0,4,1,1,0,1,1,0


In [26]:
!head -n2 {raw}/sample_submission.csv

id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
!wc -l {submissions}/0200-naive.csv

   60981 submissions/0200-naive.csv


In [28]:
!wc -l {raw}/sample_submission.csv

   60981 raw/sample_submission.csv


In [29]:
!open {submissions}

In [30]:
!open https://www.kaggle.com/c/m5-forecasting-accuracy/submit